# Westeros Tutorial: Running MESSAGE in Recursive Dynamic Mode

While considering the full time horizon in a single run is useful for a long-term energy systems planning, the recursive dynamic approach can provide insights that reflect the relevant foresight windows decision makers can have in making adaptive and robust planning.

In addition to running the model with the perfect foresights, MESSAGE can also be run using the recursive dynamic approach. Here, the model is run iteratively throughout the periods with key decision variables from the previous periods, or iterations, are fixed so that new informations do not alter decisions that are already made in the previous periods. These variables
include $CAP$ , $CAP\_NEW$, $ACT$, and $EXT$. 

In this tutorial, we will implement the recursive dynamic mode on Westeros baseline scenario. Hence, before we start, we have to make sure that we can successfully run the baseline scenarion (``westeros_baseline.ipynb``).

Let's start with importing all the libraries we need and connect to the `ixmp` platform.

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import ixmp
import message_ix

from message_ix.util import make_df
from message_ix.tools.add_tech import add_tech


%matplotlib inline

mp = ixmp.Platform("local")

## `baseline` scenario with recursive dynamic mode

In [ ]:
# cloning baseline scenario and crate 
model = "Westeros Electrified"

base = message_ix.Scenario(mp, model=model, scenario="baseline")
scen = base.clone(
    model,
    "baseline_recursive-dynamic",
    "introducing recursive dynamic mode",
    keep_solution=False,
)
scen.check_out()

year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")
country = "Westeros"

**Time to Solve the Model**

In perfect foresight mode, the solve statement we add is `scen.solve()` without any additional arguments. By default, this will tell MESSAGE to run in the perfect foreseight mode. To run MESSAGE using the recursive dynamic approach, we need to add `gams_args =["--foresight=X"]` argument to the solve statement, with `X` being the length of the foresight windows. This will pass the argument directly to `GAMS`, overiding the default values set in `MESSAGE_master.gms` and `model_setup.gms` scripts. Here, let's use `X=1` as an example. 

Similarly, technology cost learning module can be activated by setting `learningmode` value to 1. This can also be done by passing `"--learningmode=1"` GAMS argument in the solve statement, i.e., `gams_args =["--foresight=X","--learningmode=1"]`. It is important to note that the learning module lives under the recursive dynamic module. Hence, this module needs to be active if learning module is used.

In [ ]:
scen.commit(comment="Introducing recursive dynamic mode in MESSAGE")
scen.set_as_default()

scen.solve(gams_args =["--foresight=1"])

scen.var("OBJ")["lvl"]

## `emission_bound` scenario with adaptive optimization mode

In [ ]:
# cloning baseline scenario and crate 
model = "Westeros Electrified"

bound_base = message_ix.Scenario(mp, model=model, scenario="emission_bound")
bound_scen = bound_base.clone(
    model,
    "emission-bound_adaptive",
    "introducing adaptive optimisation",
    keep_solution=False,
)
bound_scen.check_out()

year_df = bound_scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = bound_scen.set("year")
country = "Westeros"

In [ ]:
bound_scen.commit(comment="Introducing recursive dynamic mode in MESSAGE")
bound_scen.set_as_default()

bound_scen.solve(gams_args = ["--foresight=3"])

bound_scen.var("OBJ")["lvl"]

## `emission_bound` scenario with adaptive optimisation and learning mode

In [ ]:
# cloning baseline scenario and crate 
learning_scen = bound_base.clone(
    model,
    "emission-bound_adaptive-learning",
    "introducing adaptive optimisation with learning",
    keep_solution=False,
)
learning_scen.check_out()

year_df = learning_scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = learning_scen.set("year")
country = "Westeros"

In [ ]:
learning_scen.set("technology")

In [ ]:
learning_pars = {
    "coal_ppl": {
        "alpha"          : 0.0,
        "beta_unit"      : 0.3,
        "beta_proj"      : 0.0,
        "gamma_unit"     : 0.04,
        "gamma_proj"     : 0.03,
        "inv_cost_refidx": 1,
        "knref_unit"     : 1,
        "sizeref_unit"   : 1,
        "sizeref_proj"   : 1,
    },
    "wind_ppl": {
        "alpha"          : 0.2,
        "beta_unit"      : 0.3,
        "beta_proj"      : 0.0,
        "gamma_unit"     : 0.03,
        "gamma_proj"     : 0.03,
        "inv_cost_refidx": 1,
        "knref_unit"     :30,
        "sizeref_unit"   : 1,
        "sizeref_proj"   :10,
    }
    
}

for key,val in learning_pars.items():
    
    learning_scen.add_set("newtec",key)
    
    for key1,val1 in val.items():
        df_par = make_df(key1,
                         technology = key,
                         value = val1,
                         unit = "-"
                        )
        learning_scen.add_par(key1,df_par)
        df_par

In [ ]:
learning_scen.par("alpha")

In [ ]:
learning_scen.commit(comment="Introducing recursive dynamic mode in MESSAGE")
learning_scen.set_as_default()

learning_scen.solve(gams_args = ["--foresight=3","--learningmode=1"])

print("OBJ without learning:", bound_scen.var("OBJ")["lvl"])
print("OBJ with learning   :", learning_scen.var("OBJ")["lvl"])

### Let's compare the results

In [ ]:
scenarios = [bound_scen, learning_scen]
labels = ["without learning", "with learning"]

for s in range(len(scenarios)):
    df = (scenarios[s].var("CAP", {"node_loc":"Westeros","technology": "wind_ppl", "year_act":act_years})
          .drop(["node_loc","technology","year_vtg"], axis=1).set_index("year_act").groupby(level=0).sum()[["lvl"]])
    tick_loc = -1.5 if s == 0 else 1.5
    plt.bar(df.index+tick_loc,df["lvl"], width=3, label=labels[s])
plt.xticks(df.index)
plt.ylabel("wind_ppl capacity")
plt.legend()
plt.show()

## With learning

## Without learning

## Parameters and variables check

## Close the connection to the database

In [ ]:
mp.close_db()